In [2]:
from time import sleep
import pandas as pd
import os
import random
from datetime import datetime, timedelta
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options
from bs4 import BeautifulSoup

url = 'https://www.reuters.com/world/africa/'
wait = 3
wait2 = 5
today = datetime.now()

from datetime import datetime, timedelta


def convert_date(input_date: str) -> str:
    try:
        now = datetime.now()
        
        #pattern1
        if "GMT" in input_date and not "," in input_date:
            parts = input_date.split()
            time_str = parts[0] + " " + parts[1]  # "7:33 PM"
            timezone_offset = parts[2]  # "GMT+9"
            
            time_obj = datetime.strptime(time_str, "%I:%M %p")
            
            tz_sign = 1 if "+" in timezone_offset else -1
            tz_hours = int(timezone_offset.split("GMT")[1])
            timezone_delta = timedelta(hours=tz_sign * tz_hours)
            
            base_date = now.replace(hour=time_obj.hour, minute=time_obj.minute, second=0, microsecond=0)
            
            adjusted_date = base_date - timezone_delta
            
            return adjusted_date.strftime("%Y-%m-%d")
        
        #pattern2
        elif "hour ago" in input_date or "hours ago" in input_date:
            hours_ago = 1 if "an hour ago" in input_date else int(input_date.split()[0])
            relative_date = now - timedelta(hours=hours_ago)
            return relative_date.strftime("%Y-%m-%d")
        
        #pattern3
        elif "," in input_date and "GMT" not in input_date:
            absolute_date = datetime.strptime(input_date, "%B %d, %Y")
            return absolute_date.strftime("%Y-%m-%d")
        
        #pattern4
        elif "," in input_date and "GMT" in input_date:
            date_part, time_part = input_date.split(" · ")
            absolute_date = datetime.strptime(date_part, "%B %d, %Y")
            time_str, timezone_offset = time_part.rsplit(" ", 1)
            time_obj = datetime.strptime(time_str, "%I:%M %p")

            tz_sign = 1 if "+" in timezone_offset else -1
            tz_hours = int(timezone_offset.split("GMT")[1])
            timezone_delta = timedelta(hours=tz_sign * tz_hours)

            #merge date and time
            full_datetime = absolute_date.replace(hour=time_obj.hour, minute=time_obj.minute)

            #UTC
            adjusted_datetime = full_datetime - timezone_delta

            return adjusted_datetime.strftime("%Y-%m-%d")
        
        return "Error: Unrecognized date format"
    except Exception as e:
        return f"Error: {e}"


error = False
chrome_check = False
try:
    options = Options()
    options.add_argument('--headless')
    driver = webdriver.Chrome(options=options)

    driver.get("about:blank")

    driver.implicitly_wait(30)
    chrome_check = True
    print('Browser opens')
except Exception as e:
    print(e)
    print('Browser does not open')
    chrome_check = False

news_selector = 'div[data-testid="MediaStoryCard"]'
title_selector = 'div h3 a'
subtitle_selector = 'p'
date_selector = 'time'
category_selector = 'div span a'
link_selector = 'a'

if chrome_check:
    driver.get(url)
    sleep(random.uniform(wait, wait2))

    d_list = []
    load_more_attempts = 2 
    for _ in range(load_more_attempts):
        soup = BeautifulSoup(driver.page_source, 'lxml')
        news_tags = soup.select(news_selector)
        for news_tag in news_tags:
            title_tag = news_tag.select(title_selector)
            subtitle_tag = news_tag.select(subtitle_selector)
            date_tag = news_tag.select(date_selector)
            category_tag = news_tag.select(category_selector)
            link_tag = news_tag.select(link_selector)

            title = title_tag[0].text if title_tag else ''
            subtitle = subtitle_tag[0].text if subtitle_tag else ''
            date = convert_date(date_tag[0].text) if date_tag else today.strftime('%Y-%m-%d')
            category = ''.join(category_tag[0].find_all(string=True, recursive=False)) if category_tag else ''
            link = f"https://www.reuters.com{link_tag[0]['href']}" if link_tag else ''

            d_list.append({
                "title": title,
                "subtitle": subtitle,
                "date": date,
                "category": category,
                "link": link
            })

        #click button
        try:
            load_more_button = driver.find_element(By.XPATH, "//button[@data-testid='TextButton']//span/span[text()='Load more articles']")

            load_more_button.click()
            sleep(random.uniform(wait, wait2))
        except Exception as e:
            print("No more 'Load More' button or error occurred:", e)
            break

    df_new = pd.DataFrame(d_list)
    if os.path.exists('ReutersTest.csv'):
        df_existing = pd.read_csv('ReutersTest.csv')
        df_combined = pd.concat([df_existing, df_new])
    else:
        df_combined = df_new

    df_combined = df_combined.drop_duplicates()
    df_combined.to_csv('ReutersTest.csv', index=False)
    print('Data saved to ReutersTest.csv')


driver.quit()

Browser opens
Data saved to ReutersTest.csv


In [20]:
url = 'https://www.reuters.com/world/africa/'
wait = 3
wait2 = 5
today = datetime.now()

error = False
chrome_check = False
options = Options()
driver = webdriver.Chrome(options=options)

driver.get("about:blank")

driver.implicitly_wait(30)
chrome_check = True
print('Browser opens')

soup = BeautifulSoup(driver.page_source, 'lxml')
news_tags = soup.select('section > div[data-testid="MediaStoryCard"]')
for news_tag in news_tags:
    title_tag = news_tag.select('h3[data-testid="Heading"] a[data-testid="Link"]')
    subtitle_tag = news_tag.select('p')
    date_tag = news_tag.select('time')
    category_tag = news_tag.select('div sapan a')
    link_tag = news_tag.select('a')

    title = title_tag[0].text
    subtitle = subtitle_tag[0].text
    date = date_tag[0].text
    category = category_tag[0].text
    link = f"https://www.reuters.com{link_tag[0]['href']}" if link_tag else ''

    print(title,subtitle,date,category,link)

    load_more_button = driver.find_element(By.XPATH, "//button[@data-testid='TextButton']//span/span[text()='Load more articles']")

    load_more_button.click()
    print("clicked")
    sleep(random.uniform(wait, wait2))
driver.quit()

Browser opens
